# Roadmap Epigenomics (Cell and Tissue Expression)

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://www.roadmapepigenomics.org/ <br>
Data Source Download: https://egg2.wustl.edu/roadmap/web_portal/processed_data.html#RNAseq_uni_proc

In [1]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [2]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [3]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [4]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [5]:
print('This notebook was run on:', date.today(), '\nPython version:', print('This notebook was run on:', date.today(), '\\nPython version:', sys.version))

This notebook was run on: 2020-06-29 \nPython version: 3.8.0 (default, Oct 28 2019, 16:14:01) 
[GCC 8.3.0]
This notebook was run on: 2020-06-29 
Python version: None


# Initialization

### Load Mapping Dictionaries

In [6]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

Gathering sources: 100%|██████████| 3/3 [00:09<00:00,  3.24s/it]


### Output Path

In [7]:
output_name = 'roadmap_epi'

path = 'Output/Roadmap-Epi'
if not os.path.exists(path):
    os.makedirs(path)

In [8]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the Roadmap Epigenomics RNA-Seq Dataset',
) %}

# Load Data

In [9]:
%%appyter code_exec

# This has to be done to fix a formatting issue in the input file
matrix = pd.read_csv({{FileField(
    constraint='.*\.gz$',
    name='read_counts', 
    label='RNA-Seq Read Counts (N.pc.gz)', 
    default='Input/Roadmap-Epi/57epigenomes.N.pc.gz',
    section='data')
}}, sep='\t', index_col=False).set_index('gene_id')

```python
# This has to be done to fix a formatting issue in the input file
matrix = pd.read_csv('Input/Roadmap-Epi/57epigenomes.N.pc.gz', sep='\t', index_col=False).set_index('gene_id')
```

In [10]:
matrix.head()

,E000,E003,E004,E005,E006,E007,E011,E012,E013,E016,...,E114,E116,E117,E118,E119,E120,E122,E123,E127,E128
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,3211,22232,22128,17579,14068,34738,15280,55040,12300,8538,...,24856,18,29162,30577,4700,9693,24191,153,16160,15225
ENSG00000000005,65,448,2085,0,55,1,18,36,6,165,...,0,0,0,0,0,8,0,2,0,0
ENSG00000000419,3099,7220,14016,11705,9808,11660,5277,12636,3425,4039,...,13894,15073,29788,15626,7011,18256,3747,17156,27026,22065
ENSG00000000457,579,1687,1797,6783,2270,2992,1462,4987,1081,822,...,4092,6717,7807,2096,1308,2677,1826,5670,7557,4630
ENSG00000000460,2157,7170,9927,9384,1143,7155,3670,9384,2669,3117,...,11207,13770,37145,4418,4046,3817,4855,27078,8274,5134


In [11]:
matrix.shape

(19795, 57)

## Load Sample Metadata

In [12]:
%%appyter code_exec

sample_meta = pd.read_csv({{FileField(
    constraint='.*\.txt$',
    name='sample_metadata', 
    label='Epigenomes Annotations (txt)', 
    default='Input/Roadmap-Epi/EG.name.txt',
    section='data')
}}, sep='\t', index_col=0, header=None)

```python

sample_meta = pd.read_csv('Input/Roadmap-Epi/EG.name.txt', sep='\t', index_col=0, header=None)
```

In [13]:
sample_meta.head()

,1
0,
E000,Universal_Human_Reference
E003,H1_Cell_Line
E004,H1_BMP4_Derived_Mesendoderm_Cultured_Cells
E005,H1_BMP4_Derived_Trophoblast_Cultured_Cells
E006,H1_Derived_Mesenchymal_Stem_Cells


In [14]:
sample_meta.shape

(58, 1)

## Load Gene Metadata

In [15]:
%%appyter code_exec

gene_meta = pd.read_csv({{FileField(
    constraint='.*\.txt$',
    name='gene_metadata', 
    label='ENSEMBL Annotations (txt)', 
    default='Input/Roadmap-Epi/Ensembl_v65.Gencode_v10.ENSG.gene_info.txt',
    section='data')
}}, sep='\t', index_col=0, header=None, usecols=[0, 6])

```python

gene_meta = pd.read_csv('Input/Roadmap-Epi/Ensembl_v65.Gencode_v10.ENSG.gene_info.txt', sep='\t', index_col=0, header=None, usecols=[0, 6])
```

In [16]:
gene_meta.head()

,6
0,
ENSG00000000003,TSPAN6
ENSG00000000005,TNMD
ENSG00000000419,DPM1
ENSG00000000457,SCYL3
ENSG00000000460,C1orf112


In [17]:
gene_meta.shape

(52475, 1)

# Pre-process Data

## Map Sample to Sample ID

In [18]:
d1 = sample_meta.to_dict()[1]
d2 = dict(zip(sample_meta.index, sample_meta[1]))

In [19]:
for k, v in d1.items():
    if not k in d2 or d2[k] != v:
        print('key:', k)

key: nan


In [20]:
sample_meta = sample_meta.rename_axis(None)

,1
E000,Universal_Human_Reference
E003,H1_Cell_Line
E004,H1_BMP4_Derived_Mesendoderm_Cultured_Cells
E005,H1_BMP4_Derived_Trophoblast_Cultured_Cells
E006,H1_Derived_Mesenchymal_Stem_Cells
E007,H1_Derived_Neuronal_Progenitor_Cultured_Cells
E011,hESC_Derived_CD184+_Endoderm_Cultured_Cells
E012,hESC_Derived_CD56+_Ectoderm_Cultured_Cells
E013,hESC_Derived_CD56+_Mesoderm_Cultured_Cells
E016,HUES64_Cell_Line


In [21]:
matrix.rename(columns=sample_meta.to_dict())

,E000,E003,E004,E005,E006,E007,E011,E012,E013,E016,...,E114,E116,E117,E118,E119,E120,E122,E123,E127,E128
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,3211,22232,22128,17579,14068,34738,15280,55040,12300,8538,...,24856,18,29162,30577,4700,9693,24191,153,16160,15225
ENSG00000000005,65,448,2085,0,55,1,18,36,6,165,...,0,0,0,0,0,8,0,2,0,0
ENSG00000000419,3099,7220,14016,11705,9808,11660,5277,12636,3425,4039,...,13894,15073,29788,15626,7011,18256,3747,17156,27026,22065
ENSG00000000457,579,1687,1797,6783,2270,2992,1462,4987,1081,822,...,4092,6717,7807,2096,1308,2677,1826,5670,7557,4630
ENSG00000000460,2157,7170,9927,9384,1143,7155,3670,9384,2669,3117,...,11207,13770,37145,4418,4046,3817,4855,27078,8274,5134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000259718,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000259741,1406,3073,7491,6572,9874,7807,3940,10956,2722,2087,...,28194,5931,9254,5900,5988,26944,4785,2687,21319,31605
ENSG00000259752,5,4,1,5,2,9,2,13,2,0,...,2,9,4,9,1,8,2,8,10,0


## Map Gene to Gene ID

In [22]:
matrix.rename(index=dict(zip(gene_meta.index, gene_meta[6])))
matrix.index.name = 'Gene Symbol'
matrix.head()

,E000,E003,E004,E005,E006,E007,E011,E012,E013,E016,...,E114,E116,E117,E118,E119,E120,E122,E123,E127,E128
Gene Symbol,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,3211,22232,22128,17579,14068,34738,15280,55040,12300,8538,...,24856,18,29162,30577,4700,9693,24191,153,16160,15225
ENSG00000000005,65,448,2085,0,55,1,18,36,6,165,...,0,0,0,0,0,8,0,2,0,0
ENSG00000000419,3099,7220,14016,11705,9808,11660,5277,12636,3425,4039,...,13894,15073,29788,15626,7011,18256,3747,17156,27026,22065
ENSG00000000457,579,1687,1797,6783,2270,2992,1462,4987,1081,822,...,4092,6717,7807,2096,1308,2677,1826,5670,7557,4630
ENSG00000000460,2157,7170,9927,9384,1143,7155,3670,9384,2669,3117,...,11207,13770,37145,4418,4046,3817,4855,27078,8274,5134


In [23]:
matrix.shape

(19795, 57)

## Save Unfiltered Matrix to file

In [ ]:
uf.save_data(matrix, path, output_name + '_matrix_unfiltered',
            compression='gzip', dtype=np.float32)

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
matrix = uf.map_symbols(matrix, symbol_lookup)
matrix.shape

## Merge Duplicate Genes By Rows and Duplicate Columns

In [ ]:
matrix = uf.merge(matrix, 'row')
matrix = uf.merge(matrix, 'column')
matrix.shape

## Remove Data that is More Than 95% Missing and Impute Missing Data

In [ ]:
matrix = uf.remove_impute(matrix)
matrix.head()

In [ ]:
matrix.shape

## Log2 Transform

In [ ]:
matrix = uf.log2(matrix)
matrix.head()

## Normalize Matrix (Quantile Normalize the Matrix by Column)

In [ ]:
matrix = uf.quantile_normalize(matrix)
matrix.head()

## Normalize Matrix (Z-Score the Rows)

In [ ]:
matrix = uf.zscore(matrix)
matrix.head()

## Histogram of First Sample

In [ ]:
matrix.iloc[:, 0].hist(bins=100)

## Histogram of First Gene

In [ ]:
matrix.iloc[0, :].hist(bins=100)

## Save Filtered Matrix

In [ ]:
uf.save_data(matrix, path, output_name + '_matrix_filtered', 
            ext='tsv', compression='gzip')

# Analyze Data

## Create Gene List

In [ ]:
gene_list = uf.gene_list(matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create matrix of Standardized values (values between -1, and 1)

In [ ]:
standard_matrix = uf.standardized_matrix(matrix)
standard_matrix.head()

In [ ]:
uf.save_data(standard_matrix, path, output_name + '_standard_matrix',
            ext='tsv', compression='gzip')

## Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [ ]:
plt.plot(matrix[matrix.columns[0]],
         standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

## Create Ternary Matrix

In [ ]:
ternary_matrix = uf.ternary_matrix(standard_matrix)
ternary_matrix.head()

In [ ]:
uf.save_data(ternary_matrix, path, output_name + '_ternary_matrix',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(ternary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'gene', 'down', path, output_name + '_gene_down_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

In [ ]:
uf.save_setlib(ternary_matrix, 'attribute', 'down', path, 
                             output_name + '_attribute_down_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(matrix.T, 'cosine')
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(matrix, 'cosine')
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(standard_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)